In [ ]:
! python -V

Python 3.8.10


In [24]:
#! pip3 install --upgrade pip
#! pip3 install pandas
#! pip3 install seaborn matplotlib
#! pip3 install pyarrow fastparquet
! pip3 install  scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 14.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 10.0 MB/s eta 0:00:00


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pickle as pkl
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
df_01 = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
df_02 = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')
df_01['duration'] = df_01.tpep_dropoff_datetime	 - df_01.tpep_pickup_datetime
df_02['duration'] = df_02.tpep_dropoff_datetime	 - df_02.tpep_pickup_datetime
df_01.duration = df_01.duration.apply(lambda td: td.total_seconds() / 60)
df_02.duration = df_02.duration.apply(lambda td: td.total_seconds() / 60)
#df_01 = df_01[(df_01.duration >= 1) & (df_01.duration <= 60)]
#df_02 = df_02[(df_02.duration >= 1) & (df_02.duration <= 60)]
df = pd.concat([df_01,df_02])

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,17.816667
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,8.400000
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,8.966667
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,10.033333
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,37.533333


In [5]:
#Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data
print('Shape of Jan data' + str(df_01.shape))
print('Shape of Feb data' + str(df_02.shape))
print('Shape of appended data' + str(df.shape))

Shape of Jan data(2463931, 20)
Shape of Feb data(2979431, 20)
Shape of appended data(5443362, 20)


In [6]:
#Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data
df_01.duration.std()

46.44530513776802

In [7]:
#Question 3: Fraction of the records left after dropping the outliers
df_treated = df_01[(df_01.duration >= 1) & (df_01.duration <= 60)]
print('The % rows remained after removing outliers ' + str(df_treated.shape[0]/df_01.shape[0]))

The % rows remained after removing outliers 0.9827547930522406


In [8]:
from sklearn.feature_extraction import DictVectorizer

In [9]:
#Question 4: Dimensionality after OHE
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df_treated[categorical] = df_treated[categorical].astype(str)

train_dicts = df_treated[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

/var/folders/gd/wps7v4255s32ns5tvpw3mwsw0000gn/T/ipykernel_33426/2301673867.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_treated[categorical] = df_treated[categorical].astype(str)


(2421440, 515)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [11]:
#Question 5: RMSE on train
target = 'duration'
y_train = df_treated[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190742248472

In [12]:
#Question 6: RMSE on validation
df_test = df_02[(df_02.duration >= 1) & (df_02.duration <= 60)]
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df_test[categorical] = df_test[categorical].astype(str)

test_dicts = df_test[categorical].to_dict(orient='records')
#dv = DictVectorizer()
X_test = dv.transform(test_dicts)



y_test = df_test[target].values
y_pred = lr.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)


/var/folders/gd/wps7v4255s32ns5tvpw3mwsw0000gn/T/ipykernel_33426/3553179267.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[categorical] = df_test[categorical].astype(str)


7.78640662117552